In [87]:
import torch
import numpy as np
import pandas as pd
import random as rd

In [88]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 3, 2
batch_size = 200
learning_rate = 1e-6

#weight matrix
w1 = torch.randn(N_in, N_hid, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_hid, N_out, device=device, dtype=dtype, requires_grad=True)

In [89]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/green_red/data.csv')
data_np_x = data[['x', 'y']].to_numpy()
data_np_y = data[['color']].to_numpy()

In [90]:
batchesX = np.asarray(np.array_split(data_np_x, 100))
batchesY = np.asarray(np.array_split(data_np_y, 100))

In [91]:
for epoch in range(1000):
    b = rd.randint(0, 99)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = Xtr.mm(w1).clamp(min=0).mm(w2)
    # ho provato a fare la cross-entropy ma non va, la 
    # la loss viene negativa
    loss = (-((y[Ytr].exp())/(y.exp().sum()).log())).sum()
    # questo valore qui sotto è a caso ma dovrebbe essere circa la lunghezza
    # di una batch
    loss = loss/100
    if epoch%100 == 99:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()


99 -0.3029716257047302
199 -0.38938685571567966
299 -0.3800498006253663
399 -0.28992588440977074
499 -0.2899276092340684
599 -0.39174923102654996
699 -0.30770640866763144
799 -0.3010648811918725
899 -0.21733477132642873
999 -0.31120022815829074
